In [1]:
import numpy as np 

sample = np.loadtxt("../matrix/6_random_adj_mat_9.txt")

In [2]:
sample

array([[0.        , 0.56182928, 0.44047985, 0.32644743, 0.71033334,
        0.33938534],
       [0.56182928, 0.        , 0.87221518, 0.69626131, 1.15341181,
        0.40357252],
       [0.44047985, 0.87221518, 0.        , 0.17749686, 0.28450999,
        0.482133  ],
       [0.32644743, 0.69626131, 0.17749686, 0.        , 0.4573035 ,
        0.30488284],
       [0.71033334, 1.15341181, 0.28450999, 0.4573035 , 0.        ,
        0.75659622],
       [0.33938534, 0.40357252, 0.482133  , 0.30488284, 0.75659622,
        0.        ]])

In [3]:
import heapq 

#Imported from AIMA repo -> https://github.com/aimacode/aima-python/blob/master/utils.py#L722

class PriorityQueue:
    """A Queue in which the minimum (or maximum) element (as determined by f and
    order) is returned first.
    If order is 'min', the item with minimum f(x) is
    returned first; if order is 'max', then it is the item with maximum f(x).
    Also supports dict-like lookup."""

    def __init__(self, order='min', f=lambda x: x):
        self.heap = []
        if order == 'min':
            self.f = f
        elif order == 'max':  # now item with max f(x)
            self.f = lambda x: -f(x)  # will be popped first
        else:
            raise ValueError("Order must be either 'min' or 'max'.")

    def append(self, item):
        """Insert item at its correct position."""
        heapq.heappush(self.heap, (self.f(item), item))

    def extend(self, items):
        """Insert each item in items at its correct position."""
        for item in items:
            self.append(item)

    def pop(self):
        """Pop and return the item (with min or max f(x) value)
        depending on the order."""
        if self.heap:
            return heapq.heappop(self.heap)[1]
        else:
            raise Exception('Trying to pop from empty PriorityQueue.')

    def __len__(self):
        """Return current capacity of PriorityQueue."""
        return len(self.heap)

    def __contains__(self, key):
        """Return True if the key is in PriorityQueue."""
        return any([item == key for _, item in self.heap])

    def __getitem__(self, key):
        """Returns the first value associated with key in PriorityQueue.
        Raises KeyError if key is not present."""
        for value, item in self.heap:
            if item == key:
                return value
        raise KeyError(str(key) + " is not in the priority queue")

    def __delitem__(self, key):
        """Delete the first occurrence of key."""
        try:
            del self.heap[[item == key for _, item in self.heap].index(True)]
        except ValueError:
            raise KeyError(str(key) + " is not in the priority queue")
        heapq.heapify(self.heap)


In [ ]:
from scipy.sparse.csgraph import minimum_spanning_tree

def astar(x):
    
    #h(x) (double check this later)
    def h_x(n, visited):
        indices = [idx for idx in range(len(x)) if idx not in visited and idx != n]
        h = minimum_spanning_tree(x[indices][:, indices]).sum()
        h += np.sort([i for i in x[n][indices] if i != 0])[:2].sum()
        return h


    #Init q with start + h(x)
    #Tuples are (nidx, path, visited, pcost, f(x))
    q = PriorityQueue(order = 'min', f = lambda x : x[-1]) 
    start = np.random.choice([i for i in range(len(x))])
    q.append(( int(start), [int(start)], {int(start)}, 0, h_x(start, set()) ))


     
    #Recurs until we find the best path
    while True > 0:
        if q.__len__() == 0:
            break

        (nidx, path, visited, cost, _) = q.pop()
        neighbors = [idx for idx in range(len(x)) if idx not in visited and x[nidx][idx] > 0]

        if len(neighbors) == 0:
            if len(path) == len(x) and x[nidx][start] > 0:
                return path + [int(start)], float(cost + x[nidx][start])
        
        else:
            for nextn in neighbors:
                f_x = cost + x[nidx][nextn] + h_x(nextn, visited)
                v = visited.copy(); v.add(nextn)
                q.append(
                    ( int(nextn), path + [int(nextn)], v, cost + x[nidx][nextn], f_x )
                )

    return [], float('inf') #Should never reach

In [6]:
import time 

s = time.process_time_ns()
path, cost = astar(sample)
e = time.process_time_ns()

path, cost, e - s

([5, 1, 0, 3, 2, 4, 5], 2.510452305693461, 12129732)

In [ ]:
'''
Raw: 12129732
W/ bitmask 
'''